In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import modules.parsers as parsers
import glob

In [2]:
spark = SparkSession.builder\
                     .master("spark://paul-spark:7077")\
                     .enableHiveSupport()\
                     .getOrCreate()
sc = spark.sparkContext

22/04/07 23:49:22 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 10.10.64.126 instead (on interface wlp59s0)
22/04/07 23:49:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/07 23:49:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pyspark.sql.functions as F
# Extract a URL from a string

## df_extract_url
#  * Take input of a dataframe
#  * Takes a column name to extract url from body of text
# Regex Source: https://stackoverflow.com/questions/28185064/python-infinite-loop-in-regex-to-match-url
def df_extract_url(tmp_df, tmp_col):
    return tmp_df.withColumn("url", F.regexp_extract(F.col(tmp_col), r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", 0))          


# Get domain name from URL and put it in domainname table
# Regex Source: https://stackoverflow.com/questions/25703360/regular-expression-extract-subdomain-domain
def df_parse_domainname(tmp_df, tmp_col):
    tmp_df = tmp_df.withColumn( "domain_name", F.regexp_extract(F.col(tmp_col) , r'^(?:http:\/\/|www\.|https:\/\/)([^\/]+)', 1)) 
    return tmp_df

# Get the domain name of an email from an email address
def df_parse_email(tmp_df, tmp_col):
    tmp_df = tmp_df.withColumn("email_username" , F.regexp_extract(F.col(tmp_col),   r'([^@]+)', 1)) 
    tmp_df = tmp_df.withColumn("email_domain"   , F.regexp_extract(F.col(tmp_col) ,  r'@(.*)'  , 1)) 
    return tmp_df

## Working with git repos

In [4]:
files = glob.glob('./out/git_out/**/*.json',  recursive=True)
first_file = files.pop()
git_repo_df = spark.read.json(first_file)
git_repo_df = parsers.df_parse_email(git_repo_df,  "author_email")
git_repo_df.first()
for tmp_df_path in files:
    tmp_git_repo_df = spark.read.json(tmp_df_path)
    tmp_git_repo_df = parsers.df_parse_email(tmp_git_repo_df,  "author_email")
    final_df = git_repo_df.unionByName(tmp_git_repo_df)

22/04/07 23:49:29 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (10.10.64.125 executor 0): java.io.FileNotFoundException: 
File file:/home/paul/Projects/DataEngineering/udacity-capstone/out/git_out/commits/ethereum/EIPs.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)
	at scala.collection.Iterator$$anon$10.hasNext(Itera

Py4JJavaError: An error occurred while calling o30.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (10.10.64.125 executor 0): java.io.FileNotFoundException: 
File file:/home/paul/Projects/DataEngineering/udacity-capstone/out/git_out/commits/ethereum/EIPs.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at or

In [ ]:
final_df.show(10, False) 

In [ ]:
final_df.toPandas()

## Working with reddit data

In [ ]:
text = ['My Profile: https://auth.geeksforgeeks.org/user/Chinmoy%20Lenka/articles in the portal of https://www.geeksforgeeks.org/',
        "Hello world https://www.udacity.com/", 
        "Hello World www.udacity.com", 
        "What about these URLs google.com NOPE",  
]

df = spark.createDataFrame(text, T.StringType()).toDF("text")

df = df.withColumn("url", F.regexp_extract(df.text, r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", 0))          
df = df.withColumn("domain", F.regexp_extract(df.url, r'^(?:http:\/\/|www\.|https:\/\/)([^\/]+)', 0))            
df[["url", "domain"]].show(5, False)

In [ ]:
reddit_df = spark.read.json('./data/reddit-stuff/xaa')# RC_2018-01-01.ndjson')

In [ ]:
reddit_df = df_extract_url(reddit_df, "body")

In [ ]:
reddit_df[["body", "url"]].filter( F.col("url") != "" ).show(3)

In [ ]:
reddit_df = parsers.df_parse_domainname(reddit_df, "url")

In [ ]:
reddit_df[["url", "domain_name"]].filter( F.col("domain_name") != "" ).show(5, False)

## Data Integration Time
Separately grouBy the domain names, one for reddit, one for git then perform join on the dataframes 

In [ ]:
reddit_domains_grouped = reddit_df.groupBy("domain_name").count()
reddit_domains_grouped = reddit_domains_grouped.withColumnRenamed("count", "reddit_domain_count")

In [ ]:
git_domains_grouped = git_repo_df.groupBy("email_domain").count()
git_domains_grouped = git_domains_grouped.withColumnRenamed("count", "git_domain_count")

In [ ]:
reddit_domains_grouped.show(3)

In [ ]:
git_domains_grouped.show(3)

In [ ]:
domains_list = reddit_domains_grouped[["domain_name", "reddit_domain_count"]].\
    withColumnRenamed("domain_name","domain_name").\
    union(git_domains_grouped[["email_domain", "git_domain_count"]])

In [ ]:
joined_domains = reddit_domains_grouped.alias("reddit")\
  .join(git_domains_grouped.alias("git"), F.col("git.email_domain") == F.col("reddit.domain_name"))

In [ ]:
git_domains_grouped.show()

In [ ]:
joined_domains.show()

22/04/07 23:48:10 WARN StandaloneAppClient$ClientEndpoint: Connection to paul-spark:7077 failed; waiting for master to reconnect...
22/04/07 23:48:10 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
22/04/07 23:48:10 WARN StandaloneAppClient$ClientEndpoint: Connection to paul-spark:7077 failed; waiting for master to reconnect...
22/04/07 23:48:10 ERROR TaskSchedulerImpl: Lost executor 0 on 10.10.64.125: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


In [33]:
unioned_domains = reddit_domains_grouped.unionByName(git_domains_grouped.withColumnRenamed("email_domain", "domain_name"),  allowMissingColumns=True)

In [34]:
unioned_domains.show()

+--------------------+-------------------+----------------+
|         domain_name|reddit_domain_count|git_domain_count|
+--------------------+-------------------+----------------+
|          www.nhs.uk|                 13|            null|
|            youtu.be|                 13|            null|
| pecuniaresearch.com|                 13|            null|
|      www.reddit.com|                 13|            null|
|                    |                435|            null|
|  bioshock.wikia.com|                 13|            null|
|         tokenate.io|               null|               1|
|patriciopalladino...|               null|               1|
|          aragon.one|               null|               2|
|          majoolr.io|               null|               1|
|           zoltu.net|               null|               1|
|        tutanota.com|               null|               1|
|        stanford.edu|               null|               1|
|          riseup.net|               nul

In [30]:
unioned_domains = reddit_domains_grouped.unionByName(git_domains_grouped)

AnalysisException: Cannot resolve column name "domain_name" among (email_domain, git_domain_count)